In [10]:
import numpy as np
import pygad
import time
import pandas as pd
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from tqdm import tqdm
import gc
from scipy import stats
import random

# Загрузка данных
data = pd.read_csv('houston_short_300.csv')
X = data[['Latitude', 'Longitude', 'Year Built', 'Beds', 'Baths', 'buildingSize', 'lotSize', 'PostalCode']]
y = data['Price']

# Функция проверки границ гиперпараметров
def check_bounds(solution):
    solution[0] = max(50, min(solution[0], 2000))  # n_estimators
    solution[1] = max(1, min(solution[1], 100))  # max_depth
    solution[2] = max(2, min(solution[2], 1000))  # num_leaves
    solution[3] = max(0.01, min(solution[3], 0.3))  # learning_rate
    solution[4] = max(10, min(solution[4], 1000))  # max_bin
    solution[5] = max(0.5, min(solution[5], 1.0))  # colsample_bytree
    return solution

# Функция оценки модели LGBM с использованием MSE
def lgbm_function(ga_instance, solution, solution_idx):
    solution = check_bounds(solution)  # Проверка границ значений
    
    params = {
        'n_estimators': int(solution[0]),
        'max_depth': int(solution[1]),
        'num_leaves': int(solution[2]),
        'learning_rate': solution[3],
        'max_bin': int(solution[4]),
        'colsample_bytree': solution[5]
    }
    
    model = lgb.LGBMRegressor(**params)
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error')
    mse = -np.mean(scores)
    return -mse  # Для минимизации возвращаем отрицательное значение MSE

# Функция сохранения результатов в CSV
def save_results_to_csv(df, filename):
    try:
        df.to_csv(filename, index=False, mode='a', header=not pd.io.common.file_exists(filename))
    except Exception as e:
        print(f"Ошибка сохранения в CSV: {e}")

# Количество внешних прогонов с разными гиперпараметрами
num_outer_runs = 30
num_inner_runs = 5
summary_results = []

# Название файла для сохранения результатов
filename = "pygad_houses.csv"

# Внешний цикл для разных гиперпараметров с отображением прогресса
for outer_run in tqdm(range(num_outer_runs), desc="Outer Runs Progress"):
    print(f"\n=== Внешний прогон {outer_run + 1} с новыми гиперпараметрами ===\n")
    
    hyperparams = {
        "num_parents_mating": random.randint(2, 10),  # Количество родителей для скрещивания
        "sol_per_pop": random.randint(10, 100),  # Размер популяции
        "mutation_percent_genes": random.uniform(1, 50),  # Процент генов для мутации
        "mutation_num_genes": random.randint(1, 3),  # Количество генов для мутации
        "crossover_type": random.choice(["single_point", "two_points", "uniform", "scattered"]),  # Тип кроссинговера
        "mutation_type": random.choice(["random", "swap", "scramble", "inversion"]),  # Тип мутации (исключаем adaptive)
        "mutation_by_replacement": random.choice([True, False]),  # Замена генов
        "random_mutation_min_val": random.uniform(-0.1, 0),  # Мин. значение мутации
        "random_mutation_max_val": random.uniform(0, 0.1),  # Макс. значение мутации
        "num_genes": 6  # Количество гиперпараметров для LGBM модели
    }

    results = []
    
    for run in range(num_inner_runs):
        print(f"\n=== Внутренний прогон {run + 1} ===\n")

        iterations_data = []

        # Генерация начальной популяции с проверкой границ значений
        initial_population = np.array([
            check_bounds([np.random.uniform(50, 2000),  # n_estimators
                          np.random.uniform(1, 100),  # max_depth
                          np.random.uniform(2, 1000),  # num_leaves
                          np.random.uniform(0.01, 0.3),  # learning_rate
                          np.random.uniform(10, 1000),  # max_bin
                          np.random.uniform(0.5, 1.0)])  # colsample_bytree
            for _ in range(hyperparams["sol_per_pop"])
        ])

        # Установка времени начала итерации
        start_time = time.time()

        ga_instance = pygad.GA(
            num_generations=20,  # Количество поколений
            num_parents_mating=hyperparams["num_parents_mating"],
            fitness_func=lgbm_function,  # Функция с тремя параметрами
            sol_per_pop=hyperparams["sol_per_pop"],
            num_genes=hyperparams["num_genes"],
            mutation_percent_genes=hyperparams["mutation_percent_genes"],
            crossover_type=hyperparams["crossover_type"],
            mutation_type=hyperparams["mutation_type"],
            mutation_by_replacement=hyperparams["mutation_by_replacement"],
            mutation_num_genes=hyperparams["mutation_num_genes"],
            random_mutation_min_val=hyperparams["random_mutation_min_val"],
            random_mutation_max_val=hyperparams["random_mutation_max_val"],
            initial_population=initial_population
        )

        # Запуск оптимизации
        ga_instance.run()

        # Время до окончания итерации
        end_time = time.time()
        elapsed_time = end_time - start_time

        # Лучшее решение и его пригодность
        solution, fitness, solution_idx = ga_instance.best_solution()
        mse = -fitness  # Восстановление MSE

        # Сохраняем результаты для каждой итерации
        run_data = {
            'Best MSE': mse,
            'Best Generation': ga_instance.generations_completed,
            'Best Generation Time': elapsed_time
        }
        results.append(run_data)

        del ga_instance
        gc.collect()

    df_results = pd.DataFrame(results)

    # Вычисление медиан и других характеристик
    median_mse = df_results['Best MSE'].median()
    mode_iteration = df_results['Best Generation'].mode()[0]
    median_time_to_best_gen = df_results['Best Generation Time'].median()

    summary_row = {
        'Медиана MSE': median_mse,
        'Мода лучшей итерации': mode_iteration,
        'Медиана времени до лучшей итерации': median_time_to_best_gen
    }

    for param, value in hyperparams.items():
        summary_row[param] = value

    summary_results.append(summary_row)

    # Сохранение результатов в CSV после каждого внешнего прогона
    df_summary = pd.DataFrame([summary_row])
    save_results_to_csv(df_summary, filename)

    gc.collect()

# Итоговая таблица с результатами всех внешних прогонов
df_summary = pd.DataFrame(summary_results)
print("\nИтоговая таблица с медианными значениями и гиперпараметрами:")
print(df_summary)


Outer Runs Progress:   0%|          | 0/30 [00:00<?, ?it/s]c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")



=== Внешний прогон 1 с новыми гиперпараметрами ===


=== Внутренний прогон 1 ===


=== Внутренний прогон 2 ===



c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")



=== Внутренний прогон 3 ===



c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


...


=== Внутренний прогон 4 ===



c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:832: UserWarning: The mutation_by_replacement parameter is set to True while the mutation_type parameter is not set to random but (inversion). Note that the mutation_by_replacement parameter has an effect only when mutation_type='random'.
  warnings.warn(f"The mutation_by_replacement parameter is set to True while the mutation_type parameter is not set to random but ({mutation_type}). Note that the mutation_by_replacement parameter has an effect only when mutation_type='random'.")
c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. T


=== Внутренний прогон 5 ===



c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:832: UserWarning: The mutation_by_replacement parameter is set to True while the mutation_type parameter is not set to random but (inversion). Note that the mutation_by_replacement parameter has an effect only when mutation_type='random'.
  warnings.warn(f"The mutation_by_replacement parameter is set to True while the mutation_type parameter is not set to random but ({mutation_type}). Note that the mutation_by_replacement parameter has an effect only when mutation_type='random'.")
c:\Users\oftuadmin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. T


Итоговая таблица с медианными значениями и гиперпараметрами:
      Медиана MSE  Мода лучшей итерации  Медиана времени до лучшей итерации   
0   504829.848899                    20                          314.062139  \
1   505830.399605                    20                          853.007792   
2   508306.579617                    20                          307.531459   
3   507262.887127                    20                          160.071216   
4   512044.633849                    20                          809.794387   
5   508268.225989                    20                          429.810840   
6   511599.751226                    20                          141.340936   
7   504819.415022                    20                          745.881103   
8   507317.459411                    20                          409.434868   
9   508437.409629                    20                           75.185530   
10  517175.484449                    20                           21.